## Setup and importing modules

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

from datetime import timedelta
from datetime import datetime
import scipy.stats as stats

import requests as r
import pandas as pd
import seaborn as s
import numpy as np

import holidays
ie_holidays = holidays.Ireland()

import postgres
import gmaps
import googlemaps
import json
import config
import math

import xgboost as xgb
from sklearn import metrics

from tqdm import tnrange, tqdm_notebook, tqdm


from tqdm._tqdm_notebook import tqdm_notebook as tdnot
tdnot.pandas()

import warnings
warnings.filterwarnings("ignore")

In [2]:
import importlib
importlib.reload(postgres.config)

<module 'config' from '/media/storage/College/S3/Github/DublinBus/Analytics/config.py'>

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Import Data


### Export the number of stops on each lineid for basic model.

In [ ]:
# # Read in all lineids from teh database and store in a text file.


# lineids = postgres.query("Select distinct(lineid) from combined;", tunnel=True)

# q = dict()
# for lidx in tnrange(len(lineids)):
    
#     lid = lineids[lidx]
#     q[lid[0]] = postgres.query("SELECT MAX(progrnumber) FROM combined WHERE lineid='%s';" % str(lid[0]), tunnel=True)
    
# with open("stops_per_line.txt",'w') as f:
#     f.write(json.dumps(q))
# f.closed

In [ ]:
with open("stops_per_line.txt",'r') as g:
    max_stops_per_line = json.loads(g.readlines()[0])

### Bus Data

In [ ]:
data = postgres.query("SELECT * FROM combined;", tunnel=True)
data = pd.DataFrame(data)

# data = pd.read_csv("stored_queries/combined145.csv")
# data.head()

In [ ]:
data.columns = ['dayofservice','tripid','lineid','direction','progrnumber','stopid','plannedDEP','plannedARR','actualDEP','actualARR','routeid']

In [ ]:
data.dayofservice = pd.to_datetime(data.dayofservice.loc[:])
data.lineid = data.lineid.astype('category')
data.routeid= data.routeid.astype('category')

In [ ]:
data.sort_values(by=['dayofservice','lineid','tripid','direction','progrnumber'],inplace=True)
# data.to_csv("stored_queries/combined145.csv", index=False, chunksize=500000)

### Trips information [for full route prediction]

In [ ]:
tripsdata = pd.read_csv("stored_queries/trips_df.csv")
tripsdata.head()

In [ ]:
tripsdata = tripsdata[['dayofservice', 'tripid', 'lineid', 'routeid', 'direction', 'actual_arr', 'actual_dep']]
tripsdata.dayofservice = pd.to_datetime(tripsdata.dayofservice)
tripsdata.dropna(inplace=True)

### Stop Information

In [ ]:
stops = pd.read_csv("stop_information.csv")

In [ ]:
cols = list(stops.columns)
cols[0] = 'ix'
stops.columns = cols
stops.drop(columns=cols[0], inplace=True)

stops.head()

### Weather Data

In [ ]:
weather = pd.read_csv("stored_queries/weather.csv")

weather.head()

In [ ]:
weather.count()

In [ ]:
weather.icon = weather.icon.astype('category')
weather.dayofservice = pd.to_datetime(weather.dayofservice)

### Import distances between stops data

In [ ]:
stop_distances = pd.read_csv("stored_queries/distancedata.csv", header=None)
stop_distances.columns = ['stopid','previous_stopid','distance']
stop_distances.head()

## Prepairing Data for Combining

#### Weather and leavetimes

In [ ]:
# leavetimes data
data.plannedARR = data.dayofservice + pd.to_timedelta(data.plannedARR, unit = 'seconds') # in nanoseconds
data.plannedDEP = data.dayofservice + pd.to_timedelta(data.plannedDEP, unit = 'seconds') # in nanoseconds
data.actualARR = data.dayofservice + pd.to_timedelta(data.actualARR, unit = 'seconds') # in nanoseconds
data.actualDEP = data.dayofservice + pd.to_timedelta(data.actualDEP, unit = 'seconds') # in nanoseconds

# new columns for combining
data['time_at_stop'] = data.actualDEP - data.actualARR
data['weather_merge_time'] = data.actualARR.dt.round('H') #  .dt useful


# weather data
weather.dayofservice = weather.dayofservice + pd.to_timedelta(weather.hour, unit='hour')

# new column for combining
weather['rkey'] = weather.dayofservice

#### Trips data preparation

In [ ]:
tripsdata.actual_arr = tripsdata.dayofservice + pd.to_timedelta(tripsdata.actual_arr, unit='seconds')
tripsdata.actual_dep = tripsdata.dayofservice + pd.to_timedelta(tripsdata.actual_dep, unit='seconds')
tripsdata['triplength'] = tripsdata.actual_arr - tripsdata.actual_dep
tripsdata['leavehour'] = tripsdata.actual_dep.dt.hour

In [ ]:
tripsdata.head()

In [ ]:
weather.head()

In [ ]:
data.head()

## Combining Data

### Combining weather and leavetimes

In [ ]:
combinedata = data.merge(weather[['icon','temperature','humidity','windSpeed','rain','rkey','hour']], 
                         left_on='weather_merge_time', 
                         right_on='rkey', 
                         how='left')

In [ ]:
# drop lineid as all are 145
combinedata.drop(columns=['rkey','lineid','weather_merge_time','plannedDEP','plannedARR','time_at_stop','actualDEP'], inplace=True)

### Combining trips and weather

In [ ]:
tripsdata['weather_merge_time'] = tripsdata.actual_dep.dt.round('H')

In [ ]:
combinedtrip = tripsdata.merge(weather[['icon','temperature','humidity','windSpeed','rain','rkey','hour']], 
                               left_on='weather_merge_time', 
                               right_on='rkey', 
                               how='left')

## Cleaning / Adding Additional features

### weekday vs weekend

In [ ]:
combinedata['weekend'] = combinedata.dayofservice.dt.weekday.isin([5,6])

### holidays

In [ ]:
combinedata['holiday'] = combinedata.dayofservice.apply(lambda x: x in ie_holidays)

### Remove inactive stops from data

In [ ]:
active_stopids = stops.stopid.values

# remove all inactive stops from the dataset. -> additional models that arent needed. 
combinedata = combinedata[combinedata.stopid.isin(active_stopids)]

### Pair Consecutive Stop IDs 

In [ ]:
# previous stopid
previousstops =  list(combinedata.stopid)
previousstops = np.array(previousstops[:-1]).astype(int)

# progrnumber of previous stopid
previousstops_progrnumber = list(combinedata.progrnumber)
previousstops_progrnumber = np.array(previousstops_progrnumber[:-1]).astype(int)

# Actual arrival time of previous stopid
previousstops_actualARR = list(combinedata.actualARR)
previousstops_actualARR = np.array(previousstops_actualARR[:-1])

# Delete the first row of the dataframe to shift the progrnumbers by one. 
combinedata = combinedata.iloc[1:]

In [ ]:
combinedata['previous_stopid'] = previousstops
combinedata['previous_stopARR'] = previousstops_actualARR
combinedata['previous_progrnumber'] = previousstops_progrnumber

In [ ]:
combinedata.head()

#### Dropping mis-matched progrnumbers

In [ ]:
# Dropping rows where progrnumber==1 as the first row is currently aligned with the last row of the previous tripid.
combinedata = combinedata[combinedata.progrnumber != 1]
combinedata.dropna(inplace=True);

#### Dropping non-consecutive stop combinations

In [ ]:
# recast type of integer cols from float to int. 
combinedata.previous_stopid = combinedata.previous_stopid.astype(int)
combinedata.previous_progrnumber = combinedata.previous_progrnumber.astype(int)

# make progrnumber difference column and then drop anything thats not exactly 1, removes data which skips stops. 
combinedata['progrnumber_difference'] = combinedata.progrnumber - combinedata.previous_progrnumber

# checking how many rows will be left. 
# combinedata.progrnumber_difference.value_counts()

In [ ]:
# remove non-consecutive stop pairs.
combinedata = combinedata[combinedata.progrnumber_difference==1]

# Remove additional columns added for this operantion
combinedata.drop(columns=['progrnumber','previous_progrnumber','progrnumber_difference'], inplace=True);

# ordering rows [and dropping irrelevant ones: direction, route_id]
combinedata = combinedata[['dayofservice', 'tripid','stopid', 'previous_stopid', 'actualARR', 'previous_stopARR',
                           'icon', 'temperature', 'humidity', 'windSpeed', 'rain', 'hour', 'weekend', 'holiday']]

#### Unique Stopid combinations

In [ ]:
# all unique stop combinations for a given lineid.
stop_pairs = combinedata[['stopid','previous_stopid']].drop_duplicates()

print("There are %d unique pairs of stops on line: %s" % (stop_pairs.count()[0], data.lineid.unique()[0]))

### Travel Time

In [ ]:
# convert to seconds
combinedata['travel_time'] = (combinedata.actualARR - combinedata.previous_stopARR).astype(int)/10**9

# drop any values less than 5 seconds [assumed erroneous]
combinedata = combinedata[combinedata.travel_time > 5]

In [ ]:
print("There are %d valid pairs" % combinedata.count()[0])

### Distance between stops [ === Don't run again === ]

In [ ]:
# Function to get the distance between two stops. 
# def get_distance(start, finish):
#     """
#     Distance between two (lat,lng) pairs
    
#     Inputs:
#     ================================
#     (int) start: stopid of first stop
#     (int) finish: stopid of last stop
    
#     Outputs:
#     ===============================
#     (int) the distance in metres between the stops. 
    
#     Notes:
#     ===============================
#     If there is an error, or the api fails to find the distance a value of None will be returned. 
#     """
#     try:
#         begin = (stops[stops.stopid==start ]['lat'].values[0], stops[stops.stopid==start ]['lng'].values[0])
#         end   = (stops[stops.stopid==finish]['lat'].values[0], stops[stops.stopid==finish]['lng'].values[0])

#     except Exception as e:

#         print(start, finish)
#         print(repr(e)) 
#         return None
        
#     API_key = config.dmatrix_key #enter Google Maps API key
#     gmaps = googlemaps.Client(key=API_key)
    
#     try:
#         call = gmaps.distance_matrix(begin, end, mode='walking')
    
#     except Exception as eL:
        
#         print(repr(eL))
#         return None
    
#     status = call['status']
    
#     if status=='OK':
#         return call["rows"][0]["elements"][0]['distance']['value']
    
#     else:
#         print(status)
#         return None

# distances_list = []

# for index, pair in tqdm_notebook(stop_pairs.iterrows(), total=stop_pairs.shape[0]):

# # for pair in stop_pairs.iterrows():
# #     start_stopid, finish_stopid = pair[1]

#     start_stopid, finish_stopid = pair[0], pair[1]
#     distances_list.append(get_distance(start_stopid, finish_stopid))
    
# distance_array = np.array(distances_list)

# stop_pairs['distance'] = distance_array

In [ ]:
combinedata = combinedata.merge(stop_distances, how='left', left_on=['stopid','previous_stopid'], right_on=['stopid','previous_stopid'])
combinedata.distance = combinedata.distance.astype(int)

### Average Speed

In [ ]:
# Taking average speed as distance / time (km/h)
combinedata['avgvel'] = (combinedata.distance / combinedata.travel_time) * (3600/1000)

# Note need to drop all data over 120 km/h -> erroneous data
combinedata = combinedata[combinedata.avgvel <= 120]

### Traffic variance / effects

In [ ]:
# add in later for clustering purposes
# combinedata['variance'] := score based on the two stops and effect traffic has on the travel time

combinedata[(combinedata.stopid==1476)&(combinedata.previous_stopid==4320)]['avgvel'].hist(bins=100)
plt.xlim([0,100])


# Note need to drop all data over 120 km/h -> erroneous data

### Removing outliers [=== Do this in the model training section as it will need to be done for each pair ===]

#### Based on Time Spent at stops

In [ ]:
# combinedata.time_at_stop.astype(int).apply(lambda x: x*10**-9).hist(bins=1000) # hitogram of times spent at a stop. 
# combinedata.time_at_stop = combinedata.time_at_stop.astype(int) / 10**9
# combinedata.head()

# use this one
## combinedata.travel_time.astype(int).hist(bins=10000) # hitogram of times spent at a stop. 
## plt.xlim([0,300])

# combinedata[combinedata['time_at_stop'] != 0].boxplot(column= ['time_at_stop'])
# combinedata.time_at_stop.describe().astype(int)

# Will use 3$\sigma$ as the threshold for outliers <br>
# Note: This method can fail to detect outliers because the outliers increase the standard deviation.

# # df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]

### Adding Month/Season

In [ ]:
combinedata['month'] = combinedata.dayofservice.dt.month

def set_season(x):
    winter = [11,12,1]
    autumn = [10,9,8]
    spring = [4,3,2]

    if x in winter:
        return 'Winter'
    elif x in autumn:
        return 'Autumn'
    elif x in spring:
        return 'Spring'
    else:
        return 'Summer'
    
combinedata['season'] = combinedata.dayofservice.dt.month.apply(set_season)

### Encoding categorical data

#### Season Categories

In [ ]:
combinedata.season = combinedata.season.astype('category', categories=['Summer','Spring','Autumn','Winter'])

combinedata = pd.concat([combinedata, pd.get_dummies(combinedata.season, prefix='season')], axis=1)
combinedata.drop(columns=['season'], inplace=True)

#### Icon Categories

In [ ]:
combinedata.icon = combinedata.icon.astype('category', categories=['partly-cloudy-day', 'partly-cloudy-night', 'clear-day', 'clear-night', 'rain', 'fog', 'cloudy', 'wind'])

combinedata = pd.concat([combinedata, pd.get_dummies(combinedata.icon, prefix='icon')], axis=1)
combinedata.drop(columns=['icon'], inplace=True)

### Drop all N/A values

In [ ]:
combinedata = combinedata.dropna() # drop na values. 
combinedata.dtypes

print("There are %d valid pairs" % combinedata.count()[0])

In [ ]:
combinedata.head().T

## Train Model

In [ ]:
modeldata = combinedata[['travel_time','stopid','previous_stopid',
                         'temperature','humidity', 'windSpeed', 'rain', 'hour', 'holiday', 'weekend',
                         'month','season_Winter','season_Autumn','season_Summer','season_Spring',
                         'icon_clear-day', 'icon_clear-night', 'icon_cloudy', 'icon_fog',
                         'icon_partly-cloudy-day', 'icon_partly-cloudy-night', 'icon_rain','icon_wind']]
modeldata.dtypes

In [ ]:
for row in unique_coordinates.iterrows():
    stop_A = row[1][1]
    stop_B = row[1][0]
    break
    
(stop_A, stop_B)

In [ ]:
modeldata = combinedata[combinedata.stopid==stop_B]
modeldata = modeldata[modeldata.previous_stopid==stop_A]

In [ ]:
print("There are %d rows relating to pair (%d, %d)" % (modeldata.count()[0], stop_A, stop_B))

In [ ]:
# need to put this in a loop over the pairs of stops. (unique)
target     = ['travel_time']
predictors = ['temperature','humidity', 'windSpeed', 'rain', 'hour', 'holiday', 'weekend',
              'month','season_Winter','season_Autumn','season_Summer','season_Spring',
              'icon_clear-day', 'icon_clear-night', 'icon_cloudy', 'icon_fog',
              'icon_partly-cloudy-day', 'icon_partly-cloudy-night', 'icon_rain','icon_wind']

In [ ]:
modeldata.travel_time.hist(bins=100)

In [ ]:
travel_time_deviation = modeldata.travel_time.std()

# ERRORS HERE fixed,  - no
# 2 sigma - 95% of data
modeldata = modeldata[abs(modeldata.travel_time-modeldata.travel_time.mean()) < 3*travel_time_deviation]
modeldata = modeldata[modeldata.travel_time >= 0]

In [ ]:
modeldata.travel_time.hist(bins=100)

In [ ]:
modeldata.travel_time.hist()


In [ ]:
train, test = train_test_split(modeldata, test_size = 0.3)

In [ ]:
RFM = RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=1)
RFM.fit(train[predictors], train[target])

In [ ]:
feature_importance = pd.DataFrame({'feature':predictors, 'importance': RFM.feature_importances_})

feature_importance.set_index('feature', inplace=True)
feature_importance.plot.barh(title='Feature importance')

In [ ]:
RFM_predictions = RFM.predict(test[predictors])

In [ ]:
plt.hist(RFM_predictions - test.travel_time, bins=20, density=True)


# # best fit of data
(mu, sigma) = stats.norm.fit(RFM_predictions - test.travel_time)

# # the histogram of the data
# n, bins, patches = plt.hist(RFM_predictions - test.travel_time, 2000, normed=1, facecolor='green', alpha=0.75)

# add a 'best fit' line
# y = mlab.normpdf(bins, mu, sigma)
l = plt.plot(bins, y, 'r--', linewidth=2)
plt.show()

print(sigma, mu)

test.travel_time.hist(bins=100)

## Evaluate Model

In [ ]:
# RMSE
trips_145_FILTERED = trips_145.dropna()

rmse_arrival_full = np.sqrt(metrics.mean_squared_error(trips_145_FILTERED.planned_arr.astype(int), trips_145_FILTERED.actual_arr.astype(int)))
rmse_depart_full  = np.sqrt(metrics.mean_squared_error(trips_145_FILTERED.planned_dep.astype(int), trips_145_FILTERED.actual_dep.astype(int)))

average_trip = (trips_145_FILTERED.actual_arr.astype(int) - trips_145_FILTERED.actual_dep.astype(int)).mean()

print(f"""

Full Trip:
Average Trip length: {round(average_trip,2)}s [{round(average_trip/3600,2)}h]

RMSE Arrival time:   {round(rmse_arrival_full,2)}s  [{round(100*(rmse_arrival_full/average_trip),2)}%] 
RMSE Departure time: {round(rmse_depart_full,2)}s  [{round(100*(rmse_depart_full/average_trip),2)}%]

""")

## Exporting Model